In [2]:
!pip install webdriver_manager

In [18]:
!pip install -c conda-forge selenium

^C


In [3]:
!pip install chromedriver-autoinstaller

  Obtaining dependency information for chromedriver-autoinstaller from https://files.pythonhosted.org/packages/6c/ae/c9946f89b42a752b38fa33fe82676e8e8fe722f3287d9909f3bd0e11e537/chromedriver_autoinstaller-0.6.3-py3-none-any.whl.metadata
  Using cached chromedriver_autoinstaller-0.6.3-py3-none-any.whl.metadata (2.0 kB)
Using cached chromedriver_autoinstaller-0.6.3-py3-none-any.whl (7.6 kB)


In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import chromedriver_autoinstaller
import pandas as pd

In [24]:
# 사용자 에이전트를 설정
    # 어떤 종류의 브라우저로 요청을 보내는지에 대한 식별 문자열
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"

# 사용자 에이전트를 포함한 Chrome 옵션을 설정
chrome_options = Options()
chrome_options.add_argument(f"user-agent={user_agent}")
chrome_options.add_argument('--headless') # Chrome 브라우저를 화면에 표시하지 않고, 백그라운드에서 실행

# Chrome 드라이버를 설치
chromedriver_autoinstaller.install()

# Chrome 드라이버를 생성
driver = webdriver.Chrome(options=chrome_options)

In [25]:
# 크롤링할 페이지 URL '역삼 맛집'
url = "https://search.naver.com/search.naver?where=view&sm=tab_jum&query=%EC%97%AD%EC%82%BC+%EB%A7%9B%EC%A7%91"
driver.get(url)

blog_links = []

# 블로그 제목 클래스 선택
titles = driver.find_elements(By.CLASS_NAME, 'title_link')

# 상위 10개의 블로그 링크 저장
for title in titles[:10]:
    blog_links.append(title.get_attribute("href"))

In [26]:
for link in blog_links:
    
    driver.get(link)

    # iframe 확인
    try:
        iframe = driver.find_element(By.TAG_NAME, 'iframe')
        driver.switch_to.frame(iframe)

        # 페이지 소스
        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')

        # 제목
        title_element = soup.find('div', {'class': 'se-title-text'})
        title = title_element.text.strip() if title_element else "제목 없음"

        # 작성일자
        date_element = soup.find('span', {'class': 'se_publishDate'})
        publish_date = date_element.text.strip() if date_element else "작성일자 없음"

        # 작성자 ID
        author_element = soup.find('span', {'class': 'nick'})
        author_id = author_element.find('a').text.strip() if author_element else "작성자 ID 없음"
        
        # 본문
        content_element = soup.find('div', {'class': 'se-main-container'})
        content = content_element.text.strip() if content_element else "본문 없음"
        
        # 데이터 리스트에 추가
        data.append({
            "링크": link,
            "작성일자": publish_date,
            "작성자 ID": author_id,
            "제목": title,
            "내용": content
        })

        # 메인 프레임으로 전환
        driver.switch_to.default_content()
        
    except NoSuchElementException as e:
        print("iframe이 없습니다. 크롤링을 진행하지 않습니다.")

iframe이 없습니다. 크롤링을 진행하지 않습니다.


In [ ]:
# 드라이버 종료
driver.quit()

In [28]:
# 데이터를 DataFrame으로 변환
df = pd.DataFrame(data)

# Excel 파일로 저장
df.to_excel("blog_data.xlsx", index=False)